In [ ]:
import subprocess
import os
import re
import tarfile
import requests
import concurrent.futures
from bs4 import BeautifulSoup
import logging
import shutil
import json
import time
import PyPDF2
import anthropic

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_arxiv_id(url: str) -> str:
    return url.split('/')[-1] if 'arxiv.org' in url else url

def remove_latex_commands(text: str) -> str:
    text = re.sub(r'\\begin{CJK\*}\{.*?\}\{.*?\}', '', text)
    text = re.sub(r'\\end{CJK\*}', '', text)
    return text

def translate_text(text: str, paper_info: dict, chunk_size: int, target_language: str = "Korean") -> str:
    cleaned_text = remove_latex_commands(text)
    logging.debug("Sending translation request to Claude API.")

    retry_attempts = 3
    for attempt in range(retry_attempts):
        try:
            client = anthropic.Client(os.environ["ANTHROPIC_API_KEY"])
            response = client.completion(
                model="claude-3-opus-20240229",
                max_tokens_to_sample=3000,
                prompt=f"""
                LaTeX 구조와 형식을 유지하면서 {target_language}로 번역
                
                번역:
                {cleaned_text}
                """
            )

            translated_content = response['completion']
            translation_result = json.loads(translated_content)
            translation_lines = translation_result["translate"]['lines']
            translated_line_count = len(translation_lines)

            if translated_line_count != chunk_size:
                time.sleep(1)
                continue

            return ''.join(translation_lines)

        except Exception as error:
            logging.error(f"Error during translation attempt {attempt + 1}: {error}")
            if attempt == retry_attempts - 1:
                raise

    raise Exception("Translation failed after multiple attempts.")

def add_custom_font_to_tex(tex_file_path: str, font_name: str = "Noto Sans KR", mono_font_name: str = "Noto Sans KR"):
    remove_cjk_related_lines(tex_file_path)
    font_setup = rf"""
        \usepackage{{kotex}}
        \usepackage{{xeCJK}}
        \setCJKmainfont{{{font_name}}}
        \setCJKmonofont{{{mono_font_name}}}
        \xeCJKsetup{{CJKspace=true}}
        """
    try:
        with open(tex_file_path, 'r+', encoding='utf-8') as file:
            lines = file.readlines()
            for i, line in enumerate(lines):
                if line.startswith(r'\documentclass'):
                    lines.insert(i + 1, font_setup)
                    break
            file.seek(0)
            file.writelines(lines)
    except Exception as e:
        logging.error(f"Failed to add custom font: {e}")
        raise

def remove_cjk_related_lines(tex_file_path: str):
    cjk_related_keywords = [
        r'\usepackage{CJKutf8}',
        r'\usepackage{kotex}',
        r'\begin{CJK}',
        r'\end{CJK}',
        r'\CJKfamily',
        r'\CJK@',
        r'\CJKrmdefault',
        r'\CJKsfdefault',
        r'\CJKttdefault',
    ]

    try:
        with open(tex_file_path, 'r+', encoding='utf-8') as file:
            lines = file.readlines()
            new_lines = [line for line in lines if not any(keyword in line for keyword in cjk_related_keywords)]
            file.seek(0)
            file.writelines(new_lines)
            file.truncate()
    except Exception as e:
        logging.error(f"Failed to remove CJK related lines: {e}")
        raise

def process_and_translate_tex_files(directory: str, paper_info: dict, read_lines: int = 30,
                                    target_language: str = "Korean", max_parallel_tasks: int = 8):
    file_line_chunks = []
    total_chunks = 0

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".tex"):
                file_path = os.path.join(root, file)
                original_file_path = file_path + "_original"
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        lines = f.readlines()

                    with open(original_file_path, 'w', encoding='utf-8') as original_f:
                        original_f.writelines(lines)

                    chunks = chunk_lines_safely(lines, read_lines)

                    for idx, chunk in enumerate(chunks):
                        file_line_chunks.append((file_path, idx, chunk))
                    total_chunks += len(chunks)

                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.writelines(lines)

                except Exception as e:
                    logging.error(f"Error reading or writing file {file_path}: {e}")

    if total_chunks == 0:
        logging.warning("No lines to translate.")
        return

    completed_chunks = 0

    def translate_chunk(file_chunk_info):
        nonlocal completed_chunks
        file_path, chunk_idx, chunk = file_chunk_info
        try:
            formatted_chunk = [line for idx, line in enumerate(chunk)]
            translated_text = translate_text(json.dumps(formatted_chunk), paper_info, len(chunk), target_language)
        except Exception as e:
            logging.error(f"Error translating chunk in file {file_path}: {e}")
            translated_text = ''.join(chunk)

        completed_chunks += 1
        progress = (completed_chunks / total_chunks) * 100
        logging.info(f"Translation progress: {progress:.2f}% completed.")
        return (file_path, chunk_idx, translated_text)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_parallel_tasks) as executor:
        translated_pairs = list(executor.map(translate_chunk, file_line_chunks))

    file_contents = {}
    for file_path, chunk_idx, translated_chunk in translated_pairs:
        if file_path not in file_contents:
            file_contents[file_path] = []
        file_contents[file_path].append((chunk_idx, translated_chunk))

    for file_path, chunks in file_contents.items():
        sorted_chunks = sorted(chunks, key=lambda x: x[0])
        translated_content = ''.join(chunk for _, chunk in sorted_chunks)
        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(translated_content)
            logging.info(f"File translated and saved: {file_path}")
        except Exception as e:
            logging.error(f"Error writing translated content to {file_path}: {e}")

def chunk_lines_safely(lines, lines_per_chunk):
    chunks = []
    current_chunk = []
    current_line_count = 0

    for line in lines:
        current_chunk.append(line)
        current_line_count += 1

        if current_line_count >= lines_per_chunk:
            chunks.append(current_chunk)
            current_chunk = []
            current_line_count = 0

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def extract_tar_gz(tar_file_path: str, extract_to: str):
    try:
        with tarfile.open(tar_file_path, 'r:gz') as tar_ref:
            tar_ref.extractall(path=extract_to)
    except Exception as e:
        logging.error(f"Failed to extract tar.gz file: {e}")
        raise

def find_main_tex_file(directory: str) -> str:
    candidate_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".tex") and "_original" not in file:
                candidate_files.append(os.path.join(root, file))

    main_candidates = []
    for file in candidate_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                contents = f.read()
                if r'\documentclass' in contents:
                    if any(keyword in contents for keyword in [r'\begin{document}', r'\usepackage', r'\title', r'\author']):
                        main_candidates.append(file)
        except Exception as e:
            logging.error(f"Failed to read file {file}: {e}")

    if main_candidates:
        return main_candidates[0]

    if candidate_files:
        return max(candidate_files, key=os.path.getsize, default=None)

    logging.warning("No .tex files found.")
    return None

def compile_main_tex(directory: str, arxiv_id: str, font_name: str = "Noto Sans KR"):
    main_tex_path = find_main_tex_file(directory)
    if main_tex_path:
        add_custom_font_to_tex(main_tex_path, font_name)
        compile_tex_to_pdf(main_tex_path, arxiv_id, compile_twice=True)
    else:
        logging.error("Main .tex file not found. Compilation aborted.")

def compile_tex_to_pdf(tex_file_path: str, arxiv_id: str, compile_twice: bool = True):
    tex_dir = os.path.dirname(tex_file_path)
    tex_file = os.path.basename(tex_file_path)

    try:
        for _ in range(2 if compile_twice else 1):
            result = subprocess.run(
                ['xelatex', '-interaction=nonstopmode', tex_file],
                cwd=tex_dir,
                encoding='utf-8'
            )
            logging.info(f"xelatex output: {result.stdout}")
            logging.info(f"xelatex errors: {result.stderr}")

        output_pdf = os.path.join(tex_dir, tex_file.replace(".tex", ".pdf"))
        if os.path.exists(output_pdf):
            current_dir = os.getcwd()
            final_pdf_path = os.path.join(current_dir, f"{arxiv_id}.pdf")
            os.rename(output_pdf, final_pdf_path)
            logging.info(f"PDF compiled and saved as: {final_pdf_path}")
        else:
            logging.error("PDF output not found after compilation.")
    except Exception as e:
        logging.error(f"Failed to compile TeX file: {e}")
        raise

def download_arxiv_intro_and_tex(arxiv_id: str, download_dir: str, target_language: str = "Korean",
                                 font_name: str = "Noto Sans KR"):
    arxiv_url = f"https://export.arxiv.org/api/query?id_list={arxiv_id}"

    try:
        response = requests.get(arxiv_url)
        response.raise_for_status()
    except requests.RequestException as e:
        logging.error(f"Failed to fetch arXiv metadata: {e}")
        raise

    soup = BeautifulSoup(response.content, 'xml')
    entry = soup.find('entry')
    if not entry:
        logging.error("ArXiv entry not found")
        raise ValueError("ArXiv entry not found")

    paper_info = {
        "title": entry.find('title').text,
        "abstract": entry.find('summary').text
    }

    tar_url = f"https://arxiv.org/src/{arxiv_id}"
    tar_file_path = os.path.join(download_dir, f"{arxiv_id}.tar.gz")

    os.makedirs(download_dir, exist_ok=True)

    try:
        with requests.get(tar_url, stream=True) as r:
            r.raise_for_status()
            with open(tar_file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except requests.RequestException as e:
        logging.error(f"arXiv 다운 실패: {e}")
        raise

    extract_to = os.path.join(download_dir, arxiv_id)

    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)

    os.makedirs(extract_to, exist_ok=True)

    extract_tar_gz(tar_file_path, extract_to)
    process_and_translate_tex_files(extract_to, paper_info, target_language=target_language)
    compile_main_tex(extract_to, arxiv_id, font_name)

def translate_pdf(pdf_path: str, target_language: str = "Korean"):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            translated_text = ""

            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text()
                translated_chunk = translate_text(text, {}, len(text.split('\n')), target_language)
                translated_text += translated_chunk

            output_path = pdf_path.replace('.pdf', f'_translated_{target_language}.txt')
            with open(output_path, 'w', encoding='utf-8') as output_file:
                output_file.write(translated_text)

            logging.info(f"Translated PDF saved as: {output_path}")
    except Exception as e:
        logging.error(f"Error translating PDF: {e}")
        raise

if __name__ == "__main__":
    arxiv_input = input("Enter ArXiv ID, URL, or local PDF path: ")
    if arxiv_input.endswith('.pdf'):
        translate_pdf(arxiv_input)
    else:
        arxiv_id = extract_arxiv_id(arxiv_input)
        download_dir = 'arxiv_downloads'
        download_arxiv_intro_and_tex(arxiv_id, download_dir)